<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Auto Suggestion project for MrBilit site
</font>
</h1>

<h2 align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Importing libraries
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>

</font>
</p>

In [1]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\quera\qenv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\quera\qenv\scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
from Levenshtein import distance

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Dataset 
    </font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    
</font>
</p>
<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>ServiceType</code>|The type of service customers need|
|<code>TypedStrings</code>|List of customer input as string sorted by time|
|<code>AcceptString</code>|The string which customer finally choosed|

</font>
</div>
</center>

<p dir=rtl style="direction: rtl; text-align: left; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3 color=red>
    <b>We are going to work on two kinds of service type</b>
</font>
<font face="vazir" size=3>
</font>
</p>

<h2 align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Reading the dataset
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
   <code dir=ltr></code>
</font>
</p>

In [9]:
data = pd.read_json('./mrbilit_search.json')
data.head()

,ServiceType,TypedStrings,AcceptString
0,flight,"[, ف, فل, فلو]",فلورانس
1,hotel,"[, H, H,, H, , آ, آو, آوا, آواپ, آواپل, آواپلا...",شیراز
2,hotel,"[, آ, آو, آوا, آواپ, آواپل, آواپلا, آواپلاس]",آواپلاس
3,hotel,[],بین المللی اسپیناس خلیج فارس
4,bus,[],تهران


In [11]:
test_data = pd.read_json('./test_data.json')
test_data.head()

,Typed
0,تهد
1,نها
2,ساوه
3,یا
4,بندر انزلی


In [12]:
cities = pd.read_csv('./iran_cities.csv')
cities.head()

,City EN,City FA,Province,Countise,District,Latitude,Longitude,Area,Elevation,2016 Census,2011 Census,Wikipedia EN,Wikipedia FA,GeoHack
0,Ab Bar,آب‌بر,زنجان,طارم,مرکزی,36.918100,48.964700,709.0,NaN,8091.0,6725.0,https://en.wikipedia.org/wiki/Ab_Bar,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%E2%...,https://geohack.toolforge.org/geohack.php?lang...
1,Ab Pakhsh,آب‌پخش,بوشهر,دشتستان,آب‌پخش,29.363706,51.070180,251204.0,7003.0,18913.0,17238.0,https://en.wikipedia.org/wiki/Ab_Pakhsh,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%E2%...,https://geohack.toolforge.org/geohack.php?lang...
2,Abad,آباد,بوشهر,تنگستان,مرکزی,29.028300,51.249100,45.0,NaN,3787.0,3787.0,"https://en.wikipedia.org/wiki/Abad,_Bushehr",https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...
3,Abadan,آبادان,خوزستان,آبادان,مرکزی,30.339167,48.304167,3.0,NaN,231476.0,212744.0,"https://en.wikipedia.org/wiki/Abadan,_Iran",https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...
4,Abadeh,آباده,فارس,آباده,مرکزی,31.180000,52.670000,1890.0,NaN,59116.0,55758.0,https://en.wikipedia.org/wiki/Abadeh,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...


In [13]:
fa2en = pd.read_csv('./typo_char.csv', index_col='FA')
fa2en.head()

,EN
FA,
ش,a
ذ,b
ز,c
ی,d
ث,e


<h2 align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Preprocessing
</font>

In [15]:
data = data.loc[data['ServiceType'].isin(['taxi', 'bus'])]
data.reset_index(drop=True, inplace=True)
data.head()

,ServiceType,TypedStrings,AcceptString
0,bus,[],تهران
1,bus,[تهران],تهران
2,bus,"[, ]",اصفهان
3,taxi,[اصفهان],اصفهان
4,bus,[تهران],تهران


In [16]:
X = []
y = []

for index, typed in enumerate(data['TypedStrings']):
    for t in typed:
        X.append(t)
        y.append(data['AcceptString'][index])

data = pd.DataFrame({'TypedStrings': X, 'AcceptString': y})
data

,TypedStrings,AcceptString
0,,تهران
1,تهران,تهران
2,,اصفهان
3,,اصفهان
4,اصفهان,اصفهان
...,...,...
75826,ب,بابل
75827,با,بابل
75828,باب,بابل
75829,بابل,بابل


In [17]:
train = pd.merge(data, cities[['City EN', 'City FA']], left_on='AcceptString', right_on='City FA', how='left')
train = train[['TypedStrings', 'AcceptString', 'City EN']]
train.head()

,TypedStrings,AcceptString,City EN
0,,تهران,Tehran
1,تهران,تهران,Tehran
2,,اصفهان,Isfahan
3,,اصفهان,Isfahan
4,اصفهان,اصفهان,Isfahan


In [18]:
for i, v in train[pd.isna(train['City EN'])]['AcceptString'].value_counts().items():
    print(i, v)

اهواز - همه پایانه‌ها 632
تهران - پایانه جنوب 390
تهران - پایانه غرب 341
شیراز - پایانه کاراندیش 252
اصفهان - پایانه کاوه 235
کنگان 208
فرودگاه امام خمینی 196
تهران - پایانه بیهقی 192
اهواز - پایانه سیاحت 157
عسلويه 137
اهواز - پایانه تپه 126
اصفهان - پایانه صفه 107
استانبول (ترکیه) 105
قم - پایانه فدک 98
تهران - همه پایانه‌ها 97
شیراز - پایانه امیرکبیر 94
سایر 78
گچساران 71
تهران - پایانه شرق 66
نورآباد 60
شیراز - پایانه مدرس 50
دیلم 45
مسجدسلیمان 44
برداسکن 41
دشتستان 35
ری 33
سلطان آباد 26
اسلام آباد غرب 24
طوالش 23
تفلیس (گرجستان) 23
اصفهان - پایانه جی 23
پایانه غرب 23
ایروان (ارمنستان) 22
ارزروم (ترکیه) 22
گرگان - همه پایانه‌ها 19
ارزوییه 17
ترابزون (ترکیه) 17
باتومی (گرجستان) 16
البرز 16
پایانه فدک 16
قم - عوارضی 15
شهر قدس 14
پایانه مرزی مهران 14
ممسنی 13
نیک شهر 12
سرچم 12
سربندر 12
دلفان 10
آنکارا (ترکیه) 9
حاجی آباد قائن 9
یزد - کمربندی 8
بویراحمد 7
فریدن 7
تکاب درگز 6
رودان 6
سیاهکلا 6
سوادکوه 6
بابا میدان 5
نجف (عراق) 5
تهران - پایانه پونک 5
اربیل (عراق) 4
قاینات 4
شهرک 4
ق

In [19]:
edit_dict = {'اهواز - همه پایانه‌ها': 'Ahvaz - All Terminals',
            'تهران - پایانه جنوب': 'Tehran - South Terminal',
            'تهران - پایانه غرب': 'Tehran - West Terminal',
            'شیراز - پایانه کاراندیش': 'Shiraz - Karandish Terminal',
            'اصفهان - پایانه کاوه': 'Isfahan - Kaveh Terminal',
            'کنگان ': 'Kangan',
            'تهران - پایانه بیهقی': 'Tehran - Biheqi Terminal',
            'اهواز - پایانه سیاحت': 'Ahvaz - Tourism Terminal',
            'عسلويه': 'Asaluyeh',
            'اهواز - پایانه تپه': 'Ahvaz - Tappeh Terminal',
            'اصفهان - پایانه صفه': 'Isfahan - Soffeh Terminal',
            'قم - پایانه فدک': 'Qom - Fadak Terminal',
            'تهران - همه پایانه‌ها': 'Tehran - All Terminals',
            'شیراز - پایانه امیرکبیر': 'Shiraz - Amir Kabir Terminal',
            'گچساران': 'Gachsaran',
            'تهران - پایانه شرق': 'Tehran - East Terminal',
            'شیراز - پایانه مدرس': 'Shiraz - Madreseh Terminal',
            'مسجدسلیمان': 'Masjed Soleyman',
            'پایانه غرب': 'Tehran - West Terminal',
            'البرز': 'Alborz',
            'تهران - پایانه پونک': 'Tehran - Ponak Terminal',
            }

In [20]:
train['City EN'] = train['City EN'].fillna(train['AcceptString'].map(edit_dict))
train

,TypedStrings,AcceptString,City EN
0,,تهران,Tehran
1,تهران,تهران,Tehran
2,,اصفهان,Isfahan
3,,اصفهان,Isfahan
4,اصفهان,اصفهان,Isfahan
...,...,...,...
76216,ب,بابل,Babol
76217,با,بابل,Babol
76218,باب,بابل,Babol
76219,بابل,بابل,Babol


In [21]:
typo = []
for accept in tqdm(train['AcceptString']):
    t = []
    for character in accept:
        if character in [' ', '-', '\u200c', '(', ')']:
            t.append(character)
        else:
            t.append(fa2en.loc[character]['EN'])
    t = ''.join(t)
    typo.append(t)

100%|██████████████████████████████████████████████████████████████████████████| 76221/76221 [00:30<00:00, 2465.89it/s]


In [22]:
train['Typo'] = typo
train.head()

,TypedStrings,AcceptString,City EN,Typo
0,,تهران,Tehran,jivhk
1,تهران,تهران,Tehran,jivhk
2,,اصفهان,Isfahan,hwtihk
3,,اصفهان,Isfahan,hwtihk
4,اصفهان,اصفهان,Isfahan,hwtihk


In [23]:
train['City EN'] = train['City EN'].str.lower()
train.head()

,TypedStrings,AcceptString,City EN,Typo
0,,تهران,tehran,jivhk
1,تهران,تهران,tehran,jivhk
2,,اصفهان,isfahan,hwtihk
3,,اصفهان,isfahan,hwtihk
4,اصفهان,اصفهان,isfahan,hwtihk


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای حل این مسئله می‌توانید از هر روشی که به ذهن‌تان می‌رسد کمک بگیرید. توجه داشته باشید که در این تمرین با یک مسئله‌ی واقعی و صنعتی سر و کار داریم، در نتیجه ممکن است برخلاف تمرین‌های رایج کالج یک راه‌حل استاندارد یا به اصطلاح سرراست برای آن وجود نداشته باشد. حتی ممکن است الگوریتم‌های یادگیری ماشین برای این مسئله به اندازه‌ی راه‌حل‌های قاعده‌ای (rule-based) یا احتمالاتی مفید واقع نشود. همچنین می‌توانید چندین روش مختلف را با همدیگر ترکیب کنید. خلاصه که ریش (🧔) و قیچی (✂️) دست شماست! هر ایده و خلاقیتی که به ذهن‌تان می‌رسد را پیاده‌سازی و آزمایش کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    چندین مثال از نمونه‌های آزمون و پیشنهادهای معقول برای آن‌ها در زیر آورده شده تا به ایده‌پردازی شما کمک کند. توجه داشته باشید در صورتی‌که پیشنهاد چندان مشابهی وجود نداشته باشد می‌توانیم از پرتکرارترین یا محتمل‌ترین برچسب‌ها استفاده می‌کنیم. یعنی در مثال زیر مشاهده می‌کنید که هرجا انتخاب بهتری نداشته‌ایم ابتدا «تهران» و سپس باقی شهرها را قرار داده‌ایم.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|Suggestion4|Suggestion3|Suggestion2|Suggestion1|Suggestion0|متن تایپ‌شده|
|:------:|:---:|:---:|:---:|:---:|:---:|
| شیراز | اصفهان | کلاله | تهران - پایانه جنوب | تهران | تهد |
| قم | قایمیه | قاینات | قائن | قائم شهر | قا |
| اصفهان | تهران | خوی | بویین ومیاندشت | بویراحمد | بوی |
| شیراز | اصفهان | تهران | یزد - کمربندی | یزد | dcn |
| شیراز | اصفهان | تهران | ماهشهر | اهواز | بندر ماهش |
| تنکابن | تهران - پایانه غرب | تهران - پایانه جنوب | تبریز | تهران | ت |
    
</font>
</div>
</center>

In [24]:
class AutoSuggest():
    def __init__(self, df, text_col, target_col, en_col, typo_col, threshold=10, max_num=10):
        target_dict = df[target_col].value_counts()
        self.target_dict = target_dict
        self.labels = list(target_dict.keys())
        self.threshold = threshold
        self.max_num = max_num
        self.vocab_size = len(df[text_col].unique())
        self.df = df
        self.text_col = text_col
        self.target_col = target_col
        self.en_col = en_col
        self.typo_col = typo_col
        self.reg = re.compile(r'[a-zA-Z,\\]')
        self.triples = self.df[[target_col, en_col, typo_col]].drop_duplicates(ignore_index=True)

    def preprocess(self, input):
        output = re.sub('\W+',' ', input)
        output = output.lower()
        return output

    # checking if the typed string is in labels
    def check(self, input):
        if input in self.labels:
            return True
        else:
            return False

    # finding all labels which starts with input
    def get_starts_with(self, input):
        results = {}
        for label, count in self.target_dict.items():
            if label.startswith(input):
                results[label] = count
        return results

    # finding all labels which contains the input
    def get_contains(self, input):
        results = {}
        for label, count in self.target_dict.items():
            if count >= self.threshold:
                if input in label:
                    results[label] = count
        return results

    # finindg all labels which are at distance of 1 to the input
    def get_one_distance(self, input):
        results = {}
        for label, count in self.target_dict.items():
            if distance(label, input) == 1:
                results[label] = count
        return results

    
    # suggest with rule-based methods
    def suggest_step1(self, input):
        suggestion = {}

        if self.check(input):
            suggestion.update({input: self.target_dict[input]})

        starts_with = self.get_starts_with(input)
        suggestion.update(starts_with)

        contains = self.get_contains(input)
        suggestion.update(contains)

        near_labels = self.get_one_distance(input)
        near_starts_with = {}
        for similar in near_labels:
            if self.check(similar):
                suggestion.update({similar: self.target_dict[similar]})

            similar_starts_with = self.get_starts_with(similar)
            near_starts_with.update(similar_starts_with)

        # sorting the near_starts_with results based on the occurance
        near_starts_with = dict(sorted(near_starts_with.items(), key=lambda item: item[1], reverse=True))
        suggestion.update(near_starts_with)

        return suggestion

    # suggest based on probabilities
    def suggest_step2(self, input, label_col):
        prob_results = {}

        df_related = self.df[self.df[self.text_col] == input]
        
        for label in self.labels:
            df_label = df_related[df_related[label_col]==label]
            likelihood = len(df_label[df_label[self.text_col] == input]) + 1
            likelihood = likelihood / (len(df_label) + self.vocab_size)
            label_prob = len(self.df[self.df[label_col] == label]) / len(self.df)
            # probability = likelihood * label_prob
            prob_results.update({label: likelihood})

        prob_results = dict(sorted(prob_results.items(), key=lambda item: item[1], reverse=True))

        return prob_results

    # suggestion for persian texts
    def suggest_fa(self, input):
        step1 = self.suggest_step1(input)
        step2 = {}

        if len(step1) < self.max_num:
            step2 = self.suggest_step2(input, self.target_col)
        
        results = {}
        results.update(step1)
        results.update(step2)

        results = list(results.keys())[:self.max_num]
        return results

    # suggestion for english texts
    def suggest_en(self, input):
        tmp_target_dict = self.target_dict
        tmp_labels = self.labels
        self.target_dict = self.df[self.en_col].value_counts()
        self.labels = list(self.target_dict.keys())

        suggestion = {}

        if self.check(input):
            suggestion.update({input: self.target_dict[input]})

        starts_with = self.get_starts_with(input)
        suggestion.update(starts_with)

        step1 = suggestion
        step2 = {}

        if len(step1) < self.max_num:
            step2 = self.suggest_step2(input, self.en_col)

        self.target_dict = tmp_target_dict
        self.labels = tmp_labels

        output1 = {}
        for key, value in step1.items():
            tmp = self.triples[self.triples[self.en_col]==key][self.target_col].values[0]
            output1[tmp] = value

        output2 = {}
        for key, value in step2.items():
            tmp = self.triples[self.triples[self.en_col]==key][self.target_col].values[0]
            output2[tmp] = value
        
        # splitting the rule-based and probability-based suggestions
        return [output1, output2]
    
    # suggestion for english texts
    def suggest_typo(self, input):
        tmp_target_dict = self.target_dict
        tmp_labels = self.labels
        self.target_dict = self.df[self.typo_col].value_counts()
        self.labels = list(self.target_dict.keys())

        suggestion = {}

        if self.check(input):
            suggestion.update({input: self.target_dict[input]})

        starts_with = self.get_starts_with(input)
        suggestion.update(starts_with)

        results = suggestion

        self.target_dict = tmp_target_dict
        self.labels = tmp_labels

        output = {}
        for key, value in results.items():
            tmp = self.triples[self.triples[self.typo_col]==key][self.target_col].values[0]
            output[tmp] = value

        return output

    def suggest(self, input):

        input = self.preprocess(input)
        
        # checking the language
        en = False
        if self.reg.match(input):
            en = True

        results = {}
        if en:
            # if it was english text
            results_en1, results_en2 = self.suggest_en(input)
            results_typo = self.suggest_typo(input)
            results.update(results_en1)
            results.update(results_typo)
            results.update(results_en2)
            results = list(results.keys())[:self.max_num]
            return results
        else:
            # if it was persian text
            return self.suggest_fa(input)


In [25]:
auto_suggest = AutoSuggest(train, 'TypedStrings', 'AcceptString', 'City EN', 'Typo', threshold=10, max_num=5)

In [26]:
auto_suggest.suggest('ت')

['تهران', 'تبریز', 'تهران - پایانه جنوب', 'تهران - پایانه غرب', 'تنکابن']

In [27]:
predictions = []
for input in tqdm(test_data['Typed']):
    predictions.append(auto_suggest.suggest(input))

100%|████████████████████████████████████████████████████████████████████████████████| 519/519 [26:24<00:00,  3.05s/it]


In [28]:
from rbo import RankingSimilarity
act = ['A', 'B', 'C']
pred = ['B', 'A', 'D']
rbo = RankingSimilarity(act, pred).rbo()
print(rbo)

0.5555555555555555


In [29]:
submission = pd.DataFrame({'Suggestion0': np.array(predictions)[:,0],
                           'Suggestion1': np.array(predictions)[:,1],
                           'Suggestion2': np.array(predictions)[:,2],
                           'Suggestion3': np.array(predictions)[:,3],
                           'Suggestion4': np.array(predictions)[:,4]})
submission.head()

,Suggestion0,Suggestion1,Suggestion2,Suggestion3,Suggestion4
0,تهران,تهران - پایانه جنوب,کلاله,اصفهان,شیراز
1,نهاوند,نکا,تهران,اصفهان,شیراز
2,ساوه,پاوه,تفرش,تهران,قم
3,یاسوج,اهواز - همه پایانه‌ها,تهران - پایانه جنوب,تهران - پایانه غرب,شیراز - پایانه کاراندیش
4,بندر انزلی,عسلویه,بندر ترکمن,تهران,اصفهان
